In [ ]:
import openai
import groq
import os
import json
from groq import Groq
from llama_index.core import SimpleDirectoryReader


In [ ]:
os.environ["GROQ_API_KEY"] = "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"

# Summarize Files


In [ ]:
reader = SimpleDirectoryReader(input_dir=".")
documents = reader.load_data()
doc_dicts = [{"content": d.text, **d.metadata} for d in documents]

In [ ]:
doc_dicts

In [ ]:
PROMPT = f"""
The following is a list of file contents, along with their metadata. For each file, provide a summary of the contents.

{doc_dicts}

Return a JSON list with the following schema:

```json
{{
  "files": [
    {{
      "filename": "name of the file",
      "summary": "summary of the content"
    }}
  ]
}}
```
""".strip()

In [ ]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Always return JSON. Do not include any other text or formatting characters.",
        },
        {
            "role": "user",
            "content": PROMPT,
        },
    ],
    model="llama3-70b-8192",
    response_format={"type": "json_object"},
)

summaries = json.loads(chat_completion.choices[0].message.content)["files"]

In [ ]:
summaries

# Create File Tree


In [ ]:
PROMPT = f"""
The following is a list of files and a summary of their contents. Read them carefully, then propose a directory structure that optimally organizes the files using known conventions and best practices.

{summaries}

You will solve this task by adding a `path` key to the JSON object below. The value of the `path` key should be the path to the file that you think is the most relevant to the summary.
""".strip()

In [ ]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Always return JSON. Do not include any other text or formatting characters.",
        },
        {
            "role": "user",
            "content": PROMPT,
        },
    ],
    model="llama3-70b-8192",
    # response_format={"type": "json_object"},
)

file_tree = json.loads(chat_completion.choices[0].message.content)

In [ ]:
import pathlib

In [ ]:
BASE_DIR = pathlib.Path("test_dir")
BASE_DIR.mkdir(exist_ok=True)

for file in file_tree:
    file["path"] = pathlib.Path(file["path"])
    # Create file in specified base directory
    (BASE_DIR / file["path"]).parent.mkdir(parents=True, exist_ok=True)
    with open(BASE_DIR / file["path"], "w") as f:
        f.write("")

In [ ]:
json.loads('{\n        "file_path": "/Users/iyaja/Git/llama-fs/LICENSE",\n        "summary": "MIT License"\n    },\n    {\n        "file_path": "/Users/iyaja/Git/llama-fs/main.py",\n        "summary": "Python script"\n    },\n    {\n        "file_path": "/Users/iyaja/Git/llama-fs/requirements.txt",\n        "summary": "List of dependencies"\n    },\n    {\n        "file_path": "/Users/iyaja/Git/llama-fs/scratch.ipynb",\n        "summary": "Jupyter notebook"\n    }')

# Test Server

In [ ]:
import requests

In [23]:
requests.post(
    "http://127.0.0.1:8000/batch",
    json={"path": "/Users/iyaja/Git/llama-fs"},
).json()

[{'src_path': 'electron-react-app/assets/icon.png',
  'dst_path': 'electron-react-app/assets/icons/icon.png',
  'summary': '\nThe image features a blue circle with a white dot in the center, surrounded by six smaller circles. The larger circle is positioned at the top left corner of the image and has a gray background. The smaller circles are evenly distributed around the central circle, creating a sense of balance within the design.'},
 {'src_path': 'electron-react-app/assets/icons/1024x1024.png',
  'dst_path': 'electron-react-app/assets/icons/1024x1024.png',
  'summary': '\nThe image features a blue circle with a white dot in the center, which is an atom. The atom has two electrons and one proton, all of which are located at the center of the circle. The entire scene is set against a black background, making the blue circle stand out prominently.'},
 {'src_path': 'electron-react-app/assets/icons/128x128.png',
  'dst_path': 'electron-react-app/assets/icons/128x128.png',
  'summary': '

# Fix the mess that is Llama Index

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter

In [ ]:
splitter = TokenTextSplitter(chunk_size=6144)

In [ ]:
reader = SimpleDirectoryReader(input_dir=".", recursive=True)
all_docs = []
for docs in reader.iter_data():
    # <do something with the documents per file>
    if len(docs) > 1:
        text = splitter.split_text("\n".join([d.text for d in docs]))[0]
        docs = [Document(text=text, metadata=docs[0].metadata)]
    all_docs.extend(docs)